In [7]:
import numpy as np
import pandas as pd
import random


# Helper functions
def check_teacher_availability(schedule, teacher, exam_day, start_hour):
    for _, value in schedule.items():
        if value[0] == teacher and value[2] == exam_day and value[3] == start_hour:
            return False
    return True

def check_teacher_consecutive_exams(schedule, teacher, exam_day):
    previous_exam = None
    for _, value in schedule.items():
        if value[0] == teacher and value[2] == exam_day:
            if previous_exam is not None:
                # Check if the current hour is 1 and the previous hour is 12
                if value[3] == 1 and previous_exam[3] == 12:
                    return False  # Consecutive exams
                # Check if the current hour is one hour ahead of the previous hour
                elif value[3] == previous_exam[3] + 1:
                    return False  # Consecutive exams
            # Check if the current hour is 9, indicating the start of the working hours
            elif value[3] < 9:
                return False  # Exam starts before 9 AM
            previous_exam = value
    return True


def generate_student_list(studentCourses):
    # Initialize an empty dictionary to store student names for each course code
    student_dict = {course: [] for course in courses}

    # Traverse studentCourse df
    for index, row in studentCourses.iterrows():
        course_code = row['Course Code']
        student_name = row['Student Name']
        # Append the student name to the list corresponding to the course code
        student_dict[course_code].append(student_name)
    return student_dict

def calculate_fitness(schedule, student_list):
    # Initialize fitness score
    fitness = 0
    
    # Apply penalty for additional constraints
    fitness += apply_additional_constraints(schedule)  
    
    # Loop through each teacher in the schedule
    for teacher in set([value[0] for value in schedule.values()]):
        # Get unique exam days for the teacher
        exam_days = set([value[2] for key, value in schedule.items() if value[0] == teacher])
        
        # Check consecutive exams and maximum exams per day constraints
        for exam_day in exam_days:
            # Penalize if teacher has non-consecutive exams
            if not check_teacher_consecutive_exams(schedule, teacher, exam_day):
                fitness -= 1
                
            # Penalize if teacher has more exams than the maximum allowed per day
            exams_on_day = [value for key, value in schedule.items() if value[0] == teacher and value[2] == exam_day]
            if len(exams_on_day) > MAX_EXAMS_PER_DAY:
                fitness -= 1
    
    return fitness


def apply_additional_constraints(schedule):
    fitness = 0
    
    # Constraint 1: No exams on Friday from 1-2
    friday_break = ('Friday', 1)
    for key, value in schedule.items():
        if value[2:] == friday_break:
            fitness -= 1
    
# Constraint 2: No student shall give more than 1 exam consecutively
    consecutive_exams = {}
    for key, value in schedule.items():
        student = key.split()[0]  # Extract student name from course code
        exam_hour = value[3]  # Extract exam hour from the schedule

        # Special case: if the exam hour is 1, adjust the previous exam hour to 12
        if exam_hour == 1:
            prev_exam_hour = 12
        else:
            prev_exam_hour = exam_hour - 1
    
        if student in consecutive_exams:
            if prev_exam_hour == 12 and exam_hour == 1:
                fitness -= 1  # Penalize fitness for consecutive exams
            elif prev_exam_hour == exam_hour - 1:
                fitness -= 1  # Penalize fitness for consecutive exams
                consecutive_exams[student] = exam_hour  # Store the current exam hour for the student

    
    # Constraint 3: Prioritize MG course exams before CS course exams for students enrolled in both
    mg_first = {}
    for key, value in schedule.items():
        student = key.split()[0]  # Extract student name from course code
        if 'MG' in key:
            mg_first[student] = value
        elif 'CS' in key:
            if student in mg_first:
                mg_exam_day = mg_first[student][2]
                cs_exam_day = value[2]
                if days.index(mg_exam_day) > days.index(cs_exam_day):
                    fitness -= 1

    # Constraint 4: Two-hour break with half the faculty free in one slot and the rest in the other
    half_faculty = len(teachers) // 2
    available_hours = [hour for hour in hours if hour not in [1, 2]]
    for day in days:
        for hour in available_hours:
            free_teachers = 0
            for teacher in teachers:
                if check_teacher_availability(schedule, teacher, day, hour):
                    free_teachers += 1
            if free_teachers >= half_faculty:
                break
        else:
            fitness -= 1

    return fitness

def generate_valid_schedule():
    while True:
        schedule = {} #initialize dictionary
        for course in courses: #generate schedule on the basis of course list
            teacher = np.random.choice(teachers) #select random teacher
            room = np.random.choice(classroom) #select random room
            exam_day = np.random.choice(days) #select random day
            start_hour = np.random.choice(hours) #select random hour
            schedule[course] = (teacher, room, exam_day, start_hour) #append all these with course as key
        
        if apply_additional_constraints(schedule): #if schedule satisfies the constraints then return the schedule else dont
            return schedule
            
def roulette_wheel_selection(population, fitness_values):
    avg_fitness = sum(fitness_values)/26  #calculate the avg of fitness values
    probabilities = [fitness /avg_fitness for fitness in fitness_values]  #calculate probabilies by dividing every fitness by avg of fitness
    rand_value = random.uniform(0, 1)  #selects a random floating point value
    cumulative_probability = 0
    for i, probability in enumerate(probabilities):
        cumulative_probability += probability  # generating number line of problablity
        if rand_value <= cumulative_probability: #if random value is within the current schedule 
            return population[i]   #then return that schedule
        

def crossover(parent1, parent2):
    crossover_point = 3  #defining a fixed crossover point
    offspring1 = {}
    offspring2 = {}
    for index, (course, details) in enumerate(parent1.items()): #iterating through parent 1
        if index < crossover_point: #if index is less than crossover point
            offspring1[course] = details #then store the details of parent1 in offspring 1
            offspring2[course] = parent2[course] #and store the details of parent2 in offspring 2
        else: #else if index is greater than crossover point
            offspring1[course] = parent2[course]#then store the details of parent1 in offspring 1
            offspring2[course] = details#and store the details of parent2 in offspring 2
    return offspring1,offspring2

def mutate(schedule):
    mutated_schedule = schedule.copy() 
    course_to_mutate = random.choice(list(mutated_schedule.keys())) #select a random course to mutate
    teacher = np.random.choice(teachers) #select random teacher
    room = np.random.choice(classroom)#select random room
    exam_day = np.random.choice(days)#select random exam day
    start_hour = np.random.choice(hours)#select random hour
    mutated_schedule[course_to_mutate] = (teacher, room, exam_day, start_hour) #edit the selected course in the whole schedule
    return mutated_schedule #and return


# Function to determine AM/PM based on hour value
def get_am_pm(hour):
    if hour in [9, 10, 11]:
        return 'AM'
    else:
        return 'PM'
    
    

def survivor_selection(population, student_list, mutation_rate):
    for _ in range(generation_count):  # Repeat survivor selection generation_count times
        # Perform constraints and calculate fitness values for the initial population
        fitness_values = [calculate_fitness(schedule, student_list) for schedule in population]
        

        # Apply roulette wheel selection
        selected_population = [roulette_wheel_selection(population, fitness_values) for _ in range(len(population))]

        # Apply crossover
        offspring_population = []  # Initialize an empty list to store offspring

        # Generate offspring by crossover
        for i in range(0, len(population), 2):
            # Select two parents for crossover
            parent1 = selected_population[i]
            parent2 = selected_population[i + 1]
        
            # Perform crossover to create two offspring
            offspring1, offspring2 = crossover(parent1, parent2)
        
            # Add the offspring to the offspring population list
            offspring_population.extend([offspring1, offspring2])

        # Apply mutation
        for idx, offspring in enumerate(offspring_population):
            # Check if mutation occurs based on mutation rate
            if random.random() < mutation_rate:
                # Perform mutation
                offspring_population[idx] = mutate(offspring)

        # Perform survivor selection
        population = offspring_population  # Replace the current population with the offspring population

    # Calculate fitness values for the last generation
    fitness_values = [calculate_fitness(schedule, student_list) for schedule in population]

    # Find the index of the schedule with the highest fitness
    best_schedule_index = fitness_values.index(max(fitness_values))

    # Select the best schedule from the last generation
    best_schedule = population[best_schedule_index]

    # Calculate fitness value for the best schedule
    best_fitness = fitness_values[best_schedule_index]

    # Convert the best schedule into a DataFrame
    best_schedule_df = pd.DataFrame(best_schedule.items(), columns=['Course', 'Details'])

    # Split the 'Details' column into separate columns
    best_schedule_df[['Teacher', 'Room', 'Day', 'Hour']] = pd.DataFrame(best_schedule_df['Details'].tolist(), index=best_schedule_df.index)

    # Drop the 'Details' column
    best_schedule_df.drop(columns=['Details'], inplace=True)

    # Convert 'Hour' column to numeric
    best_schedule_df['Hour'] = best_schedule_df['Hour'].astype(int)

    # Add 'AM/PM' column based on hour value
    best_schedule_df['AM/PM'] = best_schedule_df['Hour'].apply(get_am_pm)

    return best_schedule_df, best_fitness


# Convert the best sch

#main function

days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
hours = [9, 10, 11, 12, 1, 2, 3, 4, 5]
classroom = ['C-301', 'C-302', 'C-303', 'C-304', 'C-305', 'C-306', 'C-307', 'C-308', 'C-309', 'C-310']
COURSE_DURATION = 1
MAX_EXAMS_PER_DAY=3
population_size = 26
mutation_rate = 0.1  
generation_count=10

# Load data
data_courses = pd.read_csv('Dataset/courses.csv')
courses = data_courses['Course Code'].tolist() #convert to list

data_teachers = pd.read_csv('Dataset/teachers.csv')
teachers = data_teachers['Names'].tolist() #convert to list

data= pd.read_csv('Dataset/studentCourse.csv')
studentCourses=pd.DataFrame(data)


# Main genetic algorithm

student_list = generate_student_list(studentCourses) #generate student list

population = [generate_valid_schedule() for _ in range(population_size)] #initialize population with valid schedules

# Check constraints and calculate fitness for initial population
fitness_values = [calculate_fitness(schedule, student_list) for schedule in population]

# Initial Population
print("Initial Population:")
for index, (schedule, fitness) in enumerate(zip(population, fitness_values)):
    print("Schedule:", index + 1)
    print("Fitness Value:", fitness)  # Print the fitness value for this schedule
    for course, details in schedule.items():
        print("Course: {}, Teacher: {}, Room: {}, Day: {}, Hour: {}".format(course, details[0], details[1], details[2], details[3]))
    print()

# Apply roulette wheel selection
selected_population = [roulette_wheel_selection(population, fitness_values) for _ in range(26)]


# Selected Population
print("\nSelected Population:")
for index, schedule in enumerate(selected_population):
    fitness_value = fitness_values[index]  # Get the fitness value for this schedule
    print("Schedule:", index + 1)
    print("Fitness Value:", fitness_value)  # Print the fitness value
    for course, details in schedule.items():
        print("Course: {}, Teacher: {}, Room: {}, Day: {}, Hour: {}".format(course, details[0], details[1], details[2], details[3]))
    print()

# Apply crossover
offspring_population = []
for i in range(0, 26, 2):
    parent1 = selected_population[i]
    parent2 = selected_population[i + 1]
    offspring1, offspring2 = crossover(parent1, parent2)
    offspring_population.extend([offspring1, offspring2])

# Crossover Population
print("\nCrossover Population:")
for index, offspring in enumerate(offspring_population):
    fitness_value = calculate_fitness(offspring, student_list)  # Calculate fitness for this offspring
    print("Offspring:", index + 1)
    print("Fitness Value:", fitness_value)  # Print the fitness value
    for course, details in offspring.items():
        print("Course: {}, Teacher: {}, Room: {}, Day: {}, Hour: {}".format(course, details[0], details[1], details[2], details[3]))
    print()

# Apply mutation
for i, offspring in enumerate(offspring_population):
    if random.random() < mutation_rate:
        offspring_population[i] = mutate(offspring)

# Mutated Population
print("\nMutated Population:")
for index, offspring in enumerate(offspring_population):
    fitness_value = calculate_fitness(offspring, student_list)  # Calculate fitness for this offspring
    print("Offspring:", index + 1)
    print("Fitness Value:", fitness_value)  # Print the fitness value
    for course, details in offspring.items():
        print("Course: {}, Teacher: {}, Room: {}, Day: {}, Hour: {}".format(course, details[0], details[1], details[2], details[3]))
    print()



# Usage example:
best_schedule_df, best_fitness = survivor_selection(population, student_list, mutation_rate)
# Print the best schedule with its fitness value
print("\nBest Schedule (DataFrame) - Fitness Value:", best_fitness)
print("\nBest Schedule (DataFrame):")
print(best_schedule_df)
# Group the DataFrame by week days and sort the values within each group by 'Hour'
grouped_schedule = best_schedule_df.sort_values(by=['Day', 'Hour']).groupby('Day')

# Display the schedules for each day with corrected index
for day, schedule in grouped_schedule:
    # Reset the index starting from 1 for each group
    schedule.reset_index(drop=True, inplace=True)
    
    print(f"\n{day} Schedule:")
    print(schedule)

Initial Population:
Schedule: 1
Fitness Value: -12
Course: CS217, Teacher: Gul e Aisha, Room: C-304, Day: Friday, Hour: 2
Course: EE227, Teacher: Rohail Gulbaz, Room: C-306, Day: Thursday, Hour: 1
Course: CS211, Teacher: Sajid Khan, Room: C-302, Day: Wednesday, Hour: 11
Course: SE110, Teacher: Aqeel Shahzad, Room: C-303, Day: Friday, Hour: 10
Course: CS118, Teacher: Behjat Zuhaira, Room: C-310, Day: Monday, Hour: 11
Course: CS219, Teacher: Muhammad bin Qasim, Room: C-301, Day: Monday, Hour: 11
Course: CS220, Teacher: Ejaz Ahmed, Room: C-310, Day: Monday, Hour: 9
Course: CS302, Teacher: Hassan Raza, Room: C-310, Day: Friday, Hour: 1
Course: CY2012, Teacher: Waseem Shahzad, Room: C-301, Day: Wednesday, Hour: 9
Course: CS307, Teacher: Behjat Zuhaira, Room: C-308, Day: Tuesday, Hour: 3
Course: CS328, Teacher: Hassan Raza, Room: C-302, Day: Thursday, Hour: 3
Course: EE229, Teacher: Umair Arshad, Room: C-309, Day: Thursday, Hour: 1
Course: AI2011, Teacher: Sara Aziz, Room: C-310, Day: Thursd


Best Schedule (DataFrame) - Fitness Value: -11

Best Schedule (DataFrame):
    Course             Teacher   Room        Day  Hour AM/PM
0    CS217         Gul e Aisha  C-304     Friday     2    PM
1    EE227       Rohail Gulbaz  C-306   Thursday     1    PM
2    CS211          Sajid Khan  C-302  Wednesday    11    AM
3    SE110       Aqeel Shahzad  C-303     Friday    10    AM
4    CS118      Behjat Zuhaira  C-310     Monday    11    AM
5    CS219  Muhammad bin Qasim  C-301     Monday    11    AM
6    CS220          Ejaz Ahmed  C-310     Monday     9    AM
7    CS302         Hassan Raza  C-310     Friday     1    PM
8   CY2012      Waseem Shahzad  C-301  Wednesday     9    AM
9    CS307      Behjat Zuhaira  C-308    Tuesday     3    PM
10   CS328         Hassan Raza  C-302   Thursday     3    PM
11   EE229         Zainab Moin  C-305  Wednesday    11    AM
12  AI2011           Sara Aziz  C-310   Thursday    10    AM
13  DS3011        Umair Arshad  C-308  Wednesday     4    PM
14   CS21